<a href="https://colab.research.google.com/github/vertumno/probabilidade-analise/blob/master/Discovery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Discovering The Data

A idéia neste notebook é fazer a mineração da informação entre os diferentes banco de dados.
Para isso deveremos fazer:
- Achar o id da turma, baseado no seu nome(**Resolução de **) no dataframe de turmas_ufrn_2015_2
- Com o id da turma em mãos

In [0]:
import pandas as pd

In [39]:
disciplinas_url = 'http://bit.do/componentes_curriculares_ufrn'
disciplinas = pd.read_csv(disciplinas_url, 
                          sep=';', 
                          usecols=['id_componente','codigo','nome','unidade_responsavel'])
disciplinas.head()

,id_componente,codigo,nome,unidade_responsavel
0,18643,MUT508,COMPOSIÇÃO EM MÚSICA POPULAR III,ESCOLA DE MÚSICA
1,18644,MUT516,CO-REPETIÇÃO,ESCOLA DE MÚSICA
2,18645,MUT520,CONTRABAIXO ACÚSTICO I,ESCOLA DE MÚSICA
3,18646,MUT165,PIANO COMPLEMENTAR I,ESCOLA DE MÚSICA
4,18647,MUT167,PIANO COMPLEMENTAR III,ESCOLA DE MÚSICA


In [40]:
disciplinas[disciplinas['codigo'] == 'IMD0019']

,id_componente,codigo,nome,unidade_responsavel
21351,52416,IMD0019,MASHUPS - RECRIANDO SISTEMAS ATRAVÉS DA WEB 2.0,INSTITUTO METROPOLE DIGITAL
23695,55031,IMD0019,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,INSTITUTO METROPOLE DIGITAL


In [76]:
turmas_url = 'http://bit.do/turmas_ufrn_2015_2'
turmas = pd.read_csv(turmas_url, sep=';')
turmas.head()

,id_turma,codigo_turma,id_docente_interno,id_docente_externo,observacao,id_componente_curricular,nivel_ensino,campus_turma,local,ano,...,tipo,distancia,data_consolidacao,agrupadora,id_turma_agrupadora,qtd_aulas_lancadas,situacao_turma,convenio,modalidade_participantes,Unnamed: 25
0,57560974,01,5755580.0,NaN,NaN,55731,LATO SENSU,NaN,Caicó,2015,...,REGULAR,f,2016-02-29 16:58:56.653,f,NaN,20.0,CONSOLIDADA,NaN,Presencial,NaN
1,57560975,01,5756402.0,NaN,NaN,55770,LATO SENSU,NaN,Caicó,2015,...,REGULAR,f,2016-09-26 00:28:07.764,f,NaN,20.0,CONSOLIDADA,NaN,Presencial,NaN
2,57560976,01,22193.0,NaN,NaN,55743,LATO SENSU,NaN,Caicó,2015,...,REGULAR,f,2016-01-07 16:54:55.815,f,NaN,20.0,CONSOLIDADA,NaN,Presencial,NaN
3,57560977,01,5755386.0,NaN,NaN,55734,LATO SENSU,NaN,Caicó,2015,...,REGULAR,f,2016-01-29 17:20:44.777,f,NaN,20.0,CONSOLIDADA,NaN,Presencial,NaN
4,57560982,01,5755973.0,NaN,NaN,55735,LATO SENSU,NaN,Currais Novos,2015,...,REGULAR,f,2016-01-27 16:44:08.695,f,NaN,30.0,CONSOLIDADA,NaN,Presencial,NaN


In [79]:
rptmi_turmas = turmas[turmas['id_componente_curricular'] == 55031]

id_turma_1 = rptmi_turmas.at[3461,'id_turma']
id_turma_2 = rptmi_turmas.at[3471,'id_turma']
rptmi_turmas

,id_turma,codigo_turma,id_docente_interno,id_docente_externo,observacao,id_componente_curricular,nivel_ensino,campus_turma,local,ano,...,tipo,distancia,data_consolidacao,agrupadora,id_turma_agrupadora,qtd_aulas_lancadas,situacao_turma,convenio,modalidade_participantes,Unnamed: 25
3461,57565214,01,5756968.0,NaN,NaN,55031,GRADUAÇÃO,Campus Central,3H2 - Setor III,2015,...,REGULAR,f,2015-12-19 10:19:45.665,f,NaN,216.0,CONSOLIDADA,NaN,Presencial,NaN
3462,57565214,01,5756386.0,NaN,NaN,55031,GRADUAÇÃO,Campus Central,3H2 - Setor III,2015,...,REGULAR,f,2015-12-19 10:19:45.665,f,NaN,216.0,CONSOLIDADA,NaN,Presencial,NaN
3471,57565222,02,5757585.0,NaN,NaN,55031,GRADUAÇÃO,Campus Central,3H2 - Setor III,2015,...,REGULAR,f,2015-12-18 23:54:59.423,f,NaN,222.0,CONSOLIDADA,NaN,Presencial,NaN
3472,57565222,02,5755819.0,NaN,NaN,55031,GRADUAÇÃO,Campus Central,3H2 - Setor III,2015,...,REGULAR,f,2015-12-18 23:54:59.423,f,NaN,222.0,CONSOLIDADA,NaN,Presencial,NaN
3474,57565224,03,5757669.0,NaN,NaN,55031,GRADUAÇÃO,Campus Central,3H2 - Setor III,2015,...,REGULAR,f,2015-12-18 12:29:06.215,f,NaN,220.0,CONSOLIDADA,NaN,Presencial,NaN
3475,57565224,03,25889.0,NaN,NaN,55031,GRADUAÇÃO,Campus Central,3H2 - Setor III,2015,...,REGULAR,f,2015-12-18 12:29:06.215,f,NaN,220.0,CONSOLIDADA,NaN,Presencial,NaN


In [8]:
rptmi_turmas['capacidade_aluno']

3461    76.0
3462    76.0
3471    60.0
3472    60.0
3474    61.0
3475    61.0
Name: capacidade_aluno, dtype: float64

In [12]:
notas_turma = notas[notas['id_turma'] == id_turma_2]
notas_turma.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92 entries, 86281 to 533198
Data columns (total 11 columns):
id_turma               92 non-null int64
discente               92 non-null object
id_curso               86 non-null float64
unidade                84 non-null float64
nota                   80 non-null float64
reposicao              84 non-null object
faltas_unidade         84 non-null float64
media_final            78 non-null float64
numero_total_faltas    84 non-null float64
descricao              92 non-null object
Unnamed: 10            0 non-null float64
dtypes: float64(7), int64(1), object(3)
memory usage: 8.6+ KB


In [0]:
notas_turma.head()

,id_turma,discente,id_curso,unidade,nota,reposicao,faltas_unidade,media_final,numero_total_faltas,descricao,Unnamed: 10
115968,57565214,b433ff4854fc0247521a8d85ac933932,92127264.0,1.0,7.5,f,0.0,7.5,0.0,APROVADO,NaN
115969,57565214,b433ff4854fc0247521a8d85ac933932,92127264.0,2.0,7.0,f,0.0,7.5,0.0,APROVADO,NaN
115970,57565214,b433ff4854fc0247521a8d85ac933932,92127264.0,3.0,8.0,f,0.0,7.5,0.0,APROVADO,NaN
115971,57565214,f8fbf27b209754521cdbc40840a5a8c3,92127264.0,1.0,5.5,f,0.0,6.0,0.0,APROVADO POR NOTA,NaN
115972,57565214,f8fbf27b209754521cdbc40840a5a8c3,92127264.0,2.0,5.0,f,0.0,6.0,0.0,APROVADO POR NOTA,NaN


In [0]:

class Turmas(object):
  def __init__(self, searched_year):
    self.turmas_url = 'http://bit.do/turmas_ufrn_'+searched_year
    self.notas_url = 'http://bit.do/notas_ufrn_'+searched_year
    self.turmas = pd.DataFrame()
    self.notas = pd.DataFrame()
    self.bti_rpmti_turmas = pd.DataFrame()
   
  def read_data(self):
    self.turmas = pd.read_csv(turmas_url, sep=';')
    self.notas = pd.read_csv(notas_url, sep=';')
   
  def set_id_componente(self):
    disciplinas_url = 'http://bit.do/componentes_curriculares_ufrn'
    disciplinas = pd.read_csv(disciplinas_url, sep=';', usecols=['id_componente','codigo','nome','unidade_responsavel'])
    
    self.id_componente = int(disciplinas[disciplinas['nome'] == 'RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI']['id_componente'])
    
  def set_id_curso(self):
    cursos_url = 'http://dados.ufrn.br/dataset/02526b96-cf40-4507-90b0-3afe5ddd53e7/resource/a10bc434-9a2d-491a-ae8c-41cf643c35bc/download/cursos-de-graduacao.csv'
    cursos_url = pd.read_csv(cursos_url, sep=';')
    
    self.id_curso = int(cursos_url[cursos_url['nome'] == 'TECNOLOGIA DA INFORMAÇÃO']['id_curso'])
  
  def get_turmas_stats(self):
    self.id_turmas = self.turmas[self.turmas['id_componente_curricular'] == self.id_componente]['id_turma'].unique()
    self.all_bti_turmas = self.notas[self.notas['id_curso'] == self.id_curso]
    
    for it in self.id_turmas:
      frames = [self.all_bti_turmas[self.all_bti_turmas['id_turma'] == it]]
      self.bti_rpmti_turmas = pd.concat(frames)
      
    
  
  def get_info(self):
    print(self.turmas.info())
    print(self.notas.info())
  

In [0]:
turmas_2015 = Turmas('2015_2')
turmas_2015.read_data()

In [0]:
turmas_2015.set_id_componente()
turmas_2015.set_id_curso()
turmas_2015.get_turmas_stats()

In [137]:
turmas_2015.bti_rpmti_turmas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 174 entries, 189881 to 518901
Data columns (total 11 columns):
id_turma               174 non-null int64
discente               174 non-null object
id_curso               174 non-null float64
unidade                168 non-null float64
nota                   154 non-null float64
reposicao              168 non-null object
faltas_unidade         168 non-null float64
media_final            150 non-null float64
numero_total_faltas    168 non-null float64
descricao              174 non-null object
Unnamed: 10            0 non-null float64
dtypes: float64(7), int64(1), object(3)
memory usage: 16.3+ KB
